# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
# Import API key
from api_keys import g_key



gmaps.configure(api_key=g_key)




### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load the csv exported in Part I to a DataFrame
df = pd.read_csv('data.csv')
#print(df)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

locations = df[['lat', 'lon']]
weights = df['humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:

#A max temperature lower than 80 degrees but higher than 70.

tempBelowEighty = df[df['maxTemp'] < 26.667 ]
seventyToEighty = tempBelowEighty[tempBelowEighty['maxTemp'] > 21.1111]


#Wind speed less than 10 mph.

lowWindSpeed = seventyToEighty[seventyToEighty['windSpeed'] < 10]



#Zero cloudiness.

zeroCloudiness = lowWindSpeed[lowWindSpeed['Cloudiness'] == 0]
zeroCloudiness

#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

,City,Date,Cloudiness,Countries,humidity,lat,lon,maxTemp,windSpeed
38,Hambantota,1583882478,0,LK,83,6.12,81.12,25.78,5.49
58,São Filipe,1583882502,0,CV,73,14.90,-24.50,21.85,5.12
94,Abu Dhabi,1583882549,0,AE,63,24.47,54.37,22.22,2.60
159,Nouadhibou,1583882629,0,MR,24,20.93,-17.03,26.00,4.10
179,São João da Barra,1583882652,0,BR,83,-21.64,-41.05,24.00,4.99
226,Bandarbeyla,1583882705,0,SO,77,9.49,50.81,25.02,6.94
237,Atar,1583882717,0,MR,10,20.52,-13.05,22.15,4.97
240,Bosaso,1583882720,0,SO,73,11.28,49.18,25.33,7.69
292,Kununurra,1583882776,0,AU,88,-15.77,128.73,24.00,0.68
296,Bac Lieu,1583882780,0,VN,83,9.29,105.72,25.46,2.29


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
#fig.add_layer(markers)

lats = []
lons = []
for index, row in zeroCloudiness.iterrows():
    lats.append((row['lat']))
    lons.append((row['lon']))
    
   

    
hotels = []
for i in range(len(lats)):  

    query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=" + str(lats[i]) + "," + str(lons[i]) + "&radius=5000&type=hotel&key=" + g_key
    hotel_response = requests.get(query_url)
    hotel_json = hotel_response.json()
    hotels.append(hotel_json['results'][0])

#hotels 

hotelLats = []
hotelLongs = []


for i in hotels:
    hotelLats.append(i['geometry']['location']['lat'])
    hotelLongs.append(i['geometry']['location']['lng'])

#hotelLats
#hotelLongs

zeroCloudiness['hotelLat'] = hotelLats
zeroCloudiness['hotelLong'] = hotelLongs
                      



C:\Users\seanw\Anaconda333\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\seanw\Anaconda333\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Add marker layer ontop of heat map

marker_locations = zeroCloudiness[["hotelLat", "hotelLong"]]


markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))